# VASILY 課題

In [1]:
import os
import numpy as np
import pandas as pd
import math
import time

In [2]:
DATA_DIR = os.path.join(os.path.dirname(os.path.realpath("__file__")),"ml-100k")

今回扱っているデータの基本情報

In [3]:
ALL_USERS = 4
ALL_ITEMS = 4
ALL_RATINGS = 12

縦軸：ユーザ、横軸：映画の表に各々映画に対してユーザが評価した値が格納されているmovie_table.csvを読み込む

In [4]:
movie_table_file = "sample_table.csv"
movie_data = pd.read_csv(movie_table_file, index_col=0)
movie_data.head()

,movie1,movie2,movie3,movie4
user1,1.0,3,NaN,3.0
user2,NaN,1,3.0,NaN
user3,2.0,1,3.0,1.0
user4,1.0,3,2.0,NaN


## ピアソン相関係数を計算する

ピアソン相関係数を計算して、相関が高いものをレコメンドするようにする。ピアソン相関係数$\rho_P$は

$$
    \rho_P = \frac{E_{X,Y}[(X-\mu_X)(Y - \mu_Y)]}{\sqrt{E_X[(X - \mu_X)^2]}\sqrt{E_Y[(Y - \mu_Y)^2]}}
$$

ただし、

$$
    \mu_X = E_X[X], \mu_Y = E_Y[Y]
$$

である。

以下のような式を実装する。

* ユーザ $s$ による映画 $i$ の評価を $p_{s,i}$、ユーザ $t$ による映画 $i$ の評価を $p_{t, i}$
* ユーザ $s$ の映画の評価の平均を $E[P_s]$、ユーザ $t$ の映画の評価の平均を $E[P_t]$
* ユーザ$s$, $t$間のピアソン相関係数 $Sim_{s,t}$

$$
    Sim_{s,t} = \frac{\sum_{i=1}^{n}(p_{s,i} - E[P_s])(p_{t,i} - E[P_t])}{\sqrt{\sum_{i=1}^{n}(p_{s,i} - E[P_s])^2} \sqrt{{\sum_{i=1}^{n}(p_{t,i} - E[P_t])^2}}}
$$

ユーザの映画評価の平均を計算する

In [5]:
user1 = movie_data.ix['user1']
user1_mean = user1.mean()
print("user1 average rating = %.1f" % user1_mean)

user2 = movie_data.ix['user2']
user2_mean = user2.mean()
print("user2 average rating = %.1f" % user2_mean)

user3 = movie_data.ix['user3']
user3_mean = user3.mean()
print("user3 average rating = %.1f" % user3_mean)

user4 = movie_data.ix['user4']
user4_mean = user4.mean()
print("user4 average rating = %.1f" % user4_mean)

user1 average rating = 2.3
user2 average rating = 2.0
user3 average rating = 1.8
user4 average rating = 2.0


In [6]:
# 取り出したデータって型は何？
print(user1["movie1"])
print(type(user1["movie1"])) # numpy.float64みたい

1.0
<class 'numpy.float64'>


ピアソン相関係数を計算する関数を定義。

In [7]:
def sim_pearson(user1, user2):

    # 同じもの同士は0を返すようにする
    if user1.equals(user2):
        return 0.0
    
    user1_mean = user1.mean()
    user2_mean = user2.mean()

    sum = 0
    count_ = 0
    for i in range(1, ALL_ITEMS+1):
        movie_name = "movie%d" % i
        
        if not(math.isnan(user1[movie_name]) or math.isnan(user2[movie_name])):
            calc_ = (user1[movie_name] - user1_mean)*(user2[movie_name] - user2_mean)
            sum += calc_
            count_ += 1
    
    # 共通しているアイテムが1個以下の場合
    if count_ <= 1:
        return 0.0
    
    sos1 = 0; sos2 = 0
    for i in range(1, ALL_ITEMS+1):
        movie_name = "movie%d" % i
        
        if not(math.isnan(user1[movie_name]) or math.isnan(user2[movie_name])):
            v1 = (user1[movie_name] - user1_mean)**2
            v2 = (user2[movie_name] - user2_mean)**2

            sos1 = sos1 + v1; sos2 = sos2 + v2
            
    sqrt1 = math.sqrt(sos1)
    sqrt2 = math.sqrt(sos2)
    
    try:
        pearson_corr = sum / (sqrt1 * sqrt2)
    except ZeroDivisionError as e:
        pearson_corr = 0
    
    return pearson_corr            

In [8]:
pc21 = sim_pearson(user2, user1)
print("pearson corr(2, 1) = %f" % pc21)

pc23 = sim_pearson(user2, user3)
print("pearson corr(2, 3) = %f" % pc23)

pc24 = sim_pearson(user2, user4)
print("pearson corr(2, 4) = %f" % pc24)

pearson corr(2, 1) = 0.000000
pearson corr(2, 3) = 0.970143
pearson corr(2, 4) = -0.707107


## レコメンドアルゴリズム

評価済み映画の類似度による重み付け和を正規化したものをレコメンドスコアの指標とする。

以下のような式を実装する。

* ユーザ $s$ とユーザ $t$ とのピアソン相関係数 $Sim_{s, t}$
* ユーザ $t$ が映画 $i$ を評価した時の評価値 $p_{t, i}$
* ユーザ $s$ とユーザ $t$ が共通に評価した映画の集合 $i_{s,t}$
* $i_{s,t}$ 上での $t$ の評価値の平均 $\bar{r'_s}$
* ユーザ $s$ の評価値の平均 $\bar{r_s}$
* ユーザ $s$ が 映画 $i$ を評価した時の評価値の推定値 $Recom_{s,i}$

$$
    Recom_{s,i} = \frac{\sum_{i=1}^{n} (p_{t,i} - \bar{r'_t})Sim_{s,t}}{\sum_{i=1}^{n} |Sim_{s,t}|}
$$

まず、今回ターゲットとなる*target_user*のデータを抽出してみる。

In [9]:
target_user_name = 'user2'
target_user = movie_data.ix[target_user_name]
target_user.head(10)
#print(target_user.mean())

movie1    NaN
movie2    1.0
movie3    3.0
movie4    NaN
Name: user2, dtype: float64

*target_user*の*movie2*への推定評価値$Recom_{target\_user, movie2}$を求めてみる。

まず相関係数$\rho$を求める。*movie2*を評価済みのユーザ間で相関係数を求める必要がある。

In [10]:
target_movie = 'movie2'

for i in range(1, ALL_USERS+1):
    user_name = "user%d" % i
    user_data = movie_data.ix[user_name]
    
    # movie2を評価しているかどうか確認する
    if not(math.isnan(user_data[target_movie])):
        pc = sim_pearson(target_user, user_data)
        print("target_user = %s, user_data = %s, sim_pearson = %.1f" % (target_user_name, user_name, pc))

target_user = user2, user_data = user1, sim_pearson = 0.0
target_user = user2, user_data = user2, sim_pearson = 0.0
target_user = user2, user_data = user3, sim_pearson = 1.0
target_user = user2, user_data = user4, sim_pearson = -0.7


*taget_user*の全評価済み映画上の平均評価値を求める。

In [11]:
target_user_mean = target_user.mean()
print("%s mean rating = %.1f" % (target_user_name, target_user_mean))

user2 mean rating = 2.0


*user2*の*movie2*に対する推定評価値を計算する。

In [12]:
movie_data.head()

,movie1,movie2,movie3,movie4
user1,1.0,3,NaN,3.0
user2,NaN,1,3.0,NaN
user3,2.0,1,3.0,1.0
user4,1.0,3,2.0,NaN


In [13]:
target_movie = "movie1"

sum1 = 0
print("[Now processing numerator...]")
for i in range(1, ALL_USERS+1):
    user_name = "user%d" % i
    user_data = movie_data.ix[user_name]
    
    if not(math.isnan(user_data[target_movie])):
        pc = sim_pearson(target_user, user_data)
        print("target_user = %s, user_data = %s, sim_pearson = %.1f" % (target_user_name, user_name, pc))

        calc = pc * (user_data[target_movie] - user_data.mean())
        #print("target_user = %s, user_data = %s:" % (target_user_name, user_name))
        print("calc = %.1f * (%d - %.1f) = %.1f\n" % (pc, user_data[target_movie], user_data.mean(), calc))

        sum1 += calc

sum2 = 0
print("[Now processing denominator...]")
for i in range(1, ALL_USERS+1):
    user_name = "user%d" % i
    user_data = movie_data.ix[user_name]
    
    if not(math.isnan(user_data[target_movie])):
        pc = sim_pearson(target_user, user_data)
        abs_pc = math.fabs(pc)
        print("target_user = %s, user_data = %s, |sim_pearson| = %.1f" % (target_user_name, user_name, abs_pc))
        sum2 += abs_pc

recom = target_user_mean + sum1 / sum2
print("%s recom rating is %.1f" % (target_movie, recom))

[Now processing numerator...]
target_user = user2, user_data = user1, sim_pearson = 0.0
calc = 0.0 * (1 - 2.3) = -0.0

target_user = user2, user_data = user3, sim_pearson = 1.0
calc = 1.0 * (2 - 1.8) = 0.2

target_user = user2, user_data = user4, sim_pearson = -0.7
calc = -0.7 * (1 - 2.0) = 0.7

[Now processing denominator...]
target_user = user2, user_data = user1, |sim_pearson| = 0.0
target_user = user2, user_data = user3, |sim_pearson| = 1.0
target_user = user2, user_data = user4, |sim_pearson| = 0.7
movie1 recom rating is 2.6


## 参考

* [推薦システム 解説・講義資料](www.kamishima.net/jp/kaisetsu/)
* [推薦システムのアルゴリズム - 第9章 メモリベース型協調フィルタリング-](www.kamishima.net/archive/recsysdoc.pdf)
* [協調フィルタリングを利用した推薦システム構築](www.slideshare.net/masayuki1986/recommendation-ml)